### 1.0 Imports

In [1]:
import os
import json
import requests
import logging
import configparser
import random
from typing import Any
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, BaseTool
from langchain.llms import Ollama
from langchain_community.tools import DuckDuckGoSearchRun
#from thehive4py import TheHiveApi
#from thehive4py.api import TheHiveApi
#from thehive4py.models import *
#from thehive4py.query import *
from datetime import datetime
from random import randint
from langchain.tools import tool
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools
from langchain_core.agents import AgentFinish
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish
from textwrap import dedent
from langchain_openai import ChatOpenAI






### 2.0 Configurations

In [2]:

config = configparser.ConfigParser()
config.read('config.ini')

search_tool = DuckDuckGoSearchRun()
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
# os.environ["SERPER_API_KEY"] = "Your Key" # serper.dev API key

os.environ["OPENAI_API_KEY"] = "NA"

# Crew Agents model

morpheus = ChatOpenAI(
    model = "morpheus",
    base_url = "http://localhost:11434/v1")

# Crew Manager model

morpheus_gemma = Ollama(model="morpheus-gemma")



### 4.0 Tools

In [3]:
# class TheHiveTool(BaseTool):
#     name: str = "TheHiveTool"
#     description: str = "Enables reading and extracting data from TheHive5 Case management platform."
#     thehive_instance: TheHive = None  # Assuming TheHive is a class or type
    
#     class Config:
#         arbitrary_types_allowed = True

#     def __init__(self):
#         super().__init__()
#         config = configparser.ConfigParser()
#         config.read('config.ini')
#         self.thehive_instance = TheHive()
#         self._run("query_alerts {\"severity\": 2}")  # Run query_alerts with default severity 2

#     def _run(self, query: str) -> Any:
#         action_kwargs = self.parse_query(query)

#         try:
#             if action_kwargs['action'] == "query_alerts":
#                 return self.thehive_instance.query_alerts(severity=action_kwargs['kwargs'].get('severity'))
#             elif action_kwargs['action'] == "alerts_id":
#                 return self.thehive_instance.alerts_id(**action_kwargs['kwargs'])
#             elif action_kwargs['action'] == "create_alert":
#                 return self.thehive_instance.create_alert(**action_kwargs['kwargs'])
#             else:
#                 raise ValueError(f"Unsupported action: {action_kwargs['action']}")
#         except Exception as e:
#             logging.error(f"Error during TheHive operation: {e}")
#             return None

#     def parse_query(self, query: str) -> dict:
#         tokens = query.strip().split(" ", 1)
#         action = tokens[0].lower()
#         kwargs = {}

#         if len(tokens) > 1:
#             kwargs_str = tokens[1].strip()
#             if kwargs_str:
#                 kwargs = json.loads(kwargs_str)

#         return {"action": action, "kwargs": kwargs}

In [4]:
# if __name__ == "__main__":
#     thehive_tool = TheHiveTool()

#     # Example: Query alerts with severity 3
#     query = "query_alerts {\"severity\": 3}"
#     alerts = thehive_tool._run(query)
#     print(alerts)

    # # Example: Get alert by ID
    # query = "alerts_id {\"alert_id\": \"YourAlertID\"}"  # replace 'YourAlertID' with the actual ID
    # alert_details = thehive_tool._run(query)
    # print(alert_details)

    # # Example: Create an alert
    # query = "create_alert {\"title\": \"CrewAI Tool Alert\", \"description\": \"Create an alert with the API call to verify that the tool is working as expected\", \"severity\": 3, \"observables\": [{\"dataType\": \"ip\", \"value\": \"8.8.8.8\"}, {\"dataType\": \"url\", \"value\": \"http://example.com\"}]}"
    # new_alert = thehive_tool._run(query)
    # print(new_alert)

### 4.0 Testing the tools

### 5.0 Utilities

In [5]:
#from crewai_tools import tool

@tool("thehive_alerts")
def thehive_alerts(severity: int = 2):
    """Useful to get alerts from ThiHive Platform in json format."""
    config = configparser.ConfigParser()
    config.read('config.ini')
    # Read TheHive instance URL from the config file
    thehive_url = config['DEFAULT']['thehive_url']
    # Read TheHive API key from the config file
    thehive_api_key = config['DEFAULT']['thehive_api_key']
    if severity not in [1, 2, 3]:
        raise ValueError("Invalid severity level. Allowed values: 1 (low), 2 (medium), 3 (high)")
    
    query_params = {
        "query": [
            {
                "_name": "listAlert"
                },
            {
                "_name": "filter",
                "_field": "severity",
                "_value": severity
                },
            {
                "_name": "sort",
                "_fields": [
                    {
                        "date": "desc"
                        }
                    ]
                },
            {
                "_name": "page",
                "from": 0,
                "to": 15,
                "extraData": [
                    "importDate",
                    "caseNumber"
                    ]
                }
            ]
        }

    try:
        response = requests.post(
            f'{thehive_url}/v1/query?name=alerts',
            json=query_params,
            headers={'Authorization': f'Bearer {thehive_api_key}'}
            )
        response.raise_for_status()  # Raise for non-200 status codes
        alerts = response.json() # Convert response to JSON
        return alerts

    except requests.exceptions.HTTPError as err:
        logging.error(f"HTTP Error: {err}")
    except requests.exceptions.Timeout as err:
        logging.error(f"Request timed out: {err}")
    except requests.exceptions.RequestException as err:
        logging.error(f"General TheHive request error: {err}")

In [6]:
try:
    # Test with severity level 2 (medium)
    alerts = thehive_alerts({"severity": 3})
    #print(f"Alerts with severity level 3: {alerts}")
    #print (json.dumps(f"Alerts with severity level 3: {alerts}", indent=4))
    print(json.dumps(alerts, indent=4))
except Exception as e:
    print(f"Error occurred: {e}")


[
    {
        "_id": "~40964312",
        "_type": "Alert",
        "_createdBy": "am@onzo.local",
        "_createdAt": 1713302017593,
        "type": "AI Agents",
        "source": "CrewAI",
        "sourceRef": "628292",
        "title": "CrewAI Tool Alert",
        "description": "Create an alert with the API call to verify that the tool is working as expected",
        "severity": 3,
        "severityLabel": "HIGH",
        "date": 1713302015834,
        "tags": [],
        "tlp": 2,
        "tlpLabel": "AMBER",
        "pap": 2,
        "papLabel": "AMBER",
        "follow": true,
        "customFields": [],
        "observableCount": 0,
        "status": "New",
        "stage": "New",
        "extraData": {
            "importDate": null,
            "caseNumber": null
        },
        "newDate": 1713302015845,
        "timeToDetect": 0
    },
    {
        "_id": "~3960888",
        "_type": "Alert",
        "_createdBy": "am@onzo.local",
        "_createdAt": 1712671362261

In [7]:
agent_finishes  = []

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)

In [8]:
@tool("save_content")
def save_content(task_output):
    """Useful to save content to a markdown file"""
    print('in the save markdown tool')
    # Get today's date in the format YYYY-MM-DD
    today_date = datetime.now().strftime('%Y-%m-%d')
    # Set the filename with today's date
    filename = f"{today_date}_{randint(0,100)}.md"
    # Write the task output to the markdown file
    with open(filename, 'w') as file:
        file.write(task_output)
        # file.write(task_output.result)

    print(f"Analyzed alert saved as {filename}")

    return f"Analyzed alert saved as {filename}, please tell the user we are finished"

### 6.0 Agents

In [9]:
# Define your agents with roles and goals
soc_manager = Agent(
    role='SOC Manager',
    goal='Lead and oversee the Security Operations Center, ensuring efficient and effective monitoring, detection, and response to cyber threats across the organization.',
    backstory="""As the seasoned leader of the Security Operations Center (SOC), you have accumulated a wealth of experience in building and managing high-performing cybersecurity teams. 
    Your strategic vision and deep understanding of security operations have been instrumental in shaping the organization's defense against ever-evolving cyber threats.
    
    Overseeing the day-to-day operations of the SOC, you are responsible for ensuring that your team of skilled analysts effectively monitors and triages security alerts, 
    investigates potential incidents, and coordinates timely incident response efforts. 
    
    Your ability to make informed decisions under pressure and allocate resources effectively has proven invaluable in mitigating risks and minimizing the impact of security breaches.
    
    With a keen eye for talent, you have assembled a team of highly skilled professionals, 
    fostering an environment of continuous learning and professional development. 
    
    Your mentorship and guidance have empowered your analysts to stay ahead of the curve, constantly adapting to new threats and refining their skills.
    
    Your exceptional communication and collaboration skills enable you to bridge the gap between technical and non-technical stakeholders, 
    translating complex security information into actionable insights for executive decision-making. 
    
    You actively collaborate with cross-functional teams, such as risk management, legal, and compliance, to ensure a comprehensive and coordinated approach to cybersecurity.
    
    Beyond operational excellence, you play a pivotal role in shaping the organization's overall security strategy. Your deep understanding of industry best practices, regulatory requirements, and emerging technologies allows you to develop and implement robust security policies and frameworks. 
    Your strategic vision ensures that the organization remains resilient and prepared to tackle the ever-evolving landscape of cyber threats.
    """,
    verbose=True,
    allow_delegation=True,
    llm=morpheus,
    #max_iter=15,
    memory=True,
    #cache=True,
    step_callback=lambda x: print_agent_output(x,"SOC Manager"),
    tools=[thehive_alerts]
)


threat_intell = Agent(
    role='Threat Intelligence, Threat Hunting, and Vulnerability Expert',
    goal="Proactively identify, analyze, and mitigate potential cyber threats, vulnerabilities, and adversary tactics, techniques, and procedures (TTPs) to enhance the organization's overall security posture.",
    backstory="""As a highly skilled and experienced Threat Intelligence, Threat Hunting, and Vulnerability Expert, you are at the forefront of the organization's proactive defense against cyber threats. 
    
    Your deep understanding of the ever-evolving threat landscape, coupled with your analytical prowess, allows you to anticipate and counter potential attacks before they can cause significant damage.
    Leveraging industry-recognized frameworks like MITRE ATT&CK, you meticulously analyze and categorize adversary tactics, techniques, and procedures (TTPs), 
    enabling you to develop robust detection and mitigation strategies. 
    
    Your familiarity with the ATT&CK knowledge base empowers you to stay ahead of emerging threats and enhance the organization's defensive capabilities.
    
    With an insatiable curiosity and a passion for uncovering the latest adversary TTPs, 
    you constantly monitor and analyze various sources of threat intelligence, including dark web forums, security research publications, and industry reports. 
    
    Your ability to synthesize and contextualize this vast array of information enables you to provide actionable intelligence to the security teams, empowering them to stay one step ahead of malicious actors.
    
    As a skilled threat hunter, you employ advanced techniques and tools to actively search for indicators of compromise (IOCs) and malicious activities within the organization's networks and systems. 
    Your meticulous analysis of security logs, network traffic, and system artifacts allows you to uncover even the most sophisticated and stealthy threats, ensuring prompt detection and response.
    Your expertise in vulnerability management is equally critical. You diligently track and assess the latest disclosed vulnerabilities, prioritizing their remediation based on risk and potential impact. 
    Leveraging methodologies like OWASP Threat Modeling, you collaborate with development teams to identify and mitigate vulnerabilities in applications and systems during the design and development phases, reducing the organization's overall risk exposure.
    Your collaboration with security teams, such as incident response and patch management, ensures that vulnerabilities are promptly addressed, minimizing the organization's attack surface.
    Beyond your technical prowess, your ability to communicate complex threat and vulnerability information to stakeholders at all levels is highly valued. You leverage your strong communication skills to raise awareness, educate teams, and influence security decision-making processes, fostering a proactive and resilient security culture within the organization.
    Your commitment to continuous learning and professional development keeps you at the forefront of the ever-changing cybersecurity landscape. You actively participate in industry events, conferences, and research communities, sharing your knowledge and contributing to the advancement of threat intelligence and vulnerability management practices.
    """,
    verbose=True,
    allow_delegation=False,
    llm=morpheus,
    #max_iter=15,
    memory=True,
    #cache=True,
    step_callback=lambda x: print_agent_output(x,"Threat Intelligence, Threat Hunting, and Vulnerability Expert"),
    tools=[search_tool, save_content]
)


senior_soc_analyst = Agent(
    role='Senior Cybersecurity SOC Analyst',
    goal="Monitor and analyze security events, identify potential threats, and coordinate incident response efforts to protect the organization's systems and data.",
    backstory="""As a seasoned professional in the cybersecurity domain, you have spent years honing your skills in defending against cyber threats. 
    
    Your expertise lies in the meticulous analysis of security logs, network traffic, and system events to detect anomalies and potential intrusions.
    Working in the highly dynamic and critical environment of a Security Operations Center (SOC), you are responsible for continuously monitoring and triaging security alerts, 
    investigating suspicious activities, and orchestrating incident response efforts. Your sharp analytical mind and deep understanding of various attack vectors and threat actors 
    allow you to swiftly identify and mitigate risks before they escalate into full-blown breaches.
    
    With a keen eye for detail and a proactive approach, you collaborate closely with other security teams, such as incident response, 
    threat intelligence, and forensics, to ensure a coordinated and effective defense against cyber threats. Your ability to stay updated on the latest cybersecurity trends,
    vulnerabilities, and attack techniques is crucial in developing and implementing robust security measures.
    
    Your calm demeanor under pressure and excellent communication skills enable you to convey complex technical information to stakeholders at all levels, 
    ensuring that decision-makers are well-informed and equipped to take appropriate actions. 
    Your leadership and mentoring abilities also contribute to building a strong security culture within the organization, 
    empowering others to recognize and respond to potential threats effectively.
    """,
    llm=morpheus,
    verbose=True,
    #max_iter=15,
    memory=True,
    #cache=True,
    allow_delegation=False,
    step_callback=lambda x: print_agent_output(x,"Senior Cybersecurity SOC Analyst"),
    tools=[search_tool, save_content]
)

triage_specialist = Agent(
    role='Triage Specialist',
    goal='Efficiently evaluate and prioritize security alerts, identify potential incidents, and escalate critical issues to higher tiers for further investigation and response.',
    backstory="""As a Triage Specialist, you play a crucial role in the front line of the organization's cybersecurity defense. 
    Your keen analytical skills and attention to detail are instrumental in ensuring that security alerts and potential incidents are promptly identified, prioritized, and addressed.
    
    Operating at the Tier 1 level, you are responsible for collecting and reviewing raw data from various monitoring tools and systems. 
    Your ability to swiftly analyze alarms and alerts allows you to confirm their criticality and enrich them with relevant contextual information. 
    With a deep understanding of the organization's security landscape, you can effectively distinguish between justified alerts and false positives, mitigating the impact of alert fatigue.
    
    Your sharp eye for identifying high-risk events and potential incidents is invaluable. You meticulously evaluate and prioritize these occurrences based on their criticality, 
    ensuring that the most pressing issues receive immediate attention. 
    
    When problems cannot be resolved at your level, you seamlessly escalate them to the Tier 2 analysts, providing comprehensive documentation and context to facilitate efficient incident response.
    Beyond your analytical prowess, you also play a vital role in managing and configuring the organization's monitoring tools. Your technical expertise and attention to detail ensure that these tools are optimized for effective threat detection and monitoring, contributing to the overall efficiency of the security operations.
    
    Your ability to remain calm under pressure and communicate effectively with cross-functional teams is essential in fostering collaboration and ensuring a coordinated response to security incidents. You actively participate in knowledge-sharing sessions, 
    continuously enhancing your skills and staying updated on the latest trends and best practices in threat triage and incident response.
    Your commitment to continuous learning and professional development is driven by your passion for cybersecurity and your dedication to protecting the organization's assets and data. 
    You thrive in the fast-paced environment of the Security Operations Center, where your role as a Triage Specialist is critical in the early stages of incident detection and response.
    """,
    llm=morpheus,
    memory=True,
    #cache=True,
    verbose=True,
    allow_delegation=False,
    #max_iter=15,
    step_callback=lambda x: print_agent_output(x,"Triage Specialist"),
    tools=[save_content]
)


### 7.0 Tasks

In [10]:

# tk_thehive_input = Task(
#     description=dedent(f"""USE THE AVAILABLE TOOL and verify alerts for triage and analysis.
#                         Use the tool provided and grab the alerts in json format for triage by the Triage Specialist.
             
#                         Here's an example of how to use the tool:
                        
#                         alerts = thehive_alerts({{"severity": 2}})
    
#                         """),
# expected_output = dedent(f"""A Json file with security alerts to be analyzed.
#                            Here an example:
#                              [
#                                 {{
#                                     "_id": "~40964312",
#                                     "_type": "Alert",
#                                     "_createdBy": "am@onzo.local",
#                                     "_createdAt": 1713302017593,
#                                     "type": "AI Agents",
#                                     "source": "CrewAI",
#                                     "sourceRef": "628292",
#                                     "title": "CrewAI Tool Alert",
#                                     "description": "Create an alert with the API call to verify that the tool is working as expected",
#                                     "severity": 3,
#                                     "severityLabel": "HIGH",
#                                     "date": 1713302015834,
#                                     "tags": [],
#                                     "tlp": 2,
#                                     "tlpLabel": "AMBER",
#                                     "pap": 2,
#                                     "papLabel": "AMBER",
#                                     "follow": true,
#                                     "customFields": [],
#                                     "observableCount": 0,
#                                     "status": "New",
#                                     "stage": "New",
#                                     "extraData": {{
#                                         "importDate": null,
#                                         "caseNumber": null
#                                     }},
#                                     "newDate": 1713302015845,
#                                     "timeToDetect": 0
#                                 }},
#                                 {{
#                                     "_id": "~3960888",
#                                     "_type": "Alert",
#                                     "_createdBy": "am@onzo.local",
#                                     "_createdAt": 1712671362261,
#                                     "type": "AI Agents",
#                                     "source": "CrewAI",
#                                     "sourceRef": "234715",
#                                     "title": "CrewAI Tool Alert",
#                                     "description": "Create an alert with the API call to verify that the tool is working as expected",
#                                     "severity": 3,
#                                     "severityLabel": "HIGH",
#                                     "date": 1712671362255,
#                                     "tags": [],
#                                     "tlp": 2,
#                                     "tlpLabel": "AMBER",
#                                     "pap": 2,
#                                     "papLabel": "AMBER",
#                                     "follow": true,
#                                     "customFields": [],
#                                     "observableCount": 0,
#                                     "status": "New",
#                                     "stage": "New",
#                                     "extraData": {{
#                                         "importDate": null,
#                                         "caseNumber": null
#                                     }},
#                                     "newDate": 1712671362258,
#                                     "timeToDetect": 0
#                                 }},
#                                 {{
#                                     "_id": "~8273936",
#                                     "_type": "Alert",
#                                     "_createdBy": "am@onzo.local",
#                                     "_createdAt": 1712671246160,
#                                     "type": "test",
#                                     "source": "CrewAI",
#                                     "sourceRef": "377312",
#                                     "title": "CrewAI Tool Alert",
#                                     "description": "Create an alert with the API call to verify that the tool is working as expected",
#                                     "severity": 3,
#                                     "severityLabel": "HIGH",
#                                     "date": 1712671246142,
#                                     "tags": [],
#                                     "tlp": 2,
#                                     "tlpLabel": "AMBER",
#                                     "pap": 2,
#                                     "papLabel": "AMBER",
#                                     "follow": true,
#                                     "customFields": [],
#                                     "observableCount": 0,
#                                     "status": "New",
#                                     "stage": "New",
#                                     "extraData": {{
#                                         "importDate": null,
#                                         "caseNumber": null
#                                     }},
#                                     "newDate": 1712671246145,
#                                     "timeToDetect": 0
#                                 }},
#                                 {{
#                                     "_id": "~8163440",
#                                     "_type": "Alert",
#                                     "_createdBy": "am@onzo.local",
#                                     "_createdAt": 1712668671441,
#                                     "type": "test",
#                                     "source": "CrewAI",
#                                     "sourceRef": "1",
#                                     "title": "Test from CrewAI Tool",
#                                     "description": "Create an alert with the API call to verify that the tool is working as expected",
#                                     "severity": 3,
#                                     "severityLabel": "HIGH",
#                                     "date": 1712668671432,
#                                     "tags": [],
#                                     "tlp": 2,
#                                     "tlpLabel": "AMBER",
#                                     "pap": 2,
#                                     "papLabel": "AMBER",
#                                     "follow": true,
#                                     "customFields": [],
#                                     "observableCount": 0,
#                                     "status": "New",
#                                     "stage": "New",
#                                     "extraData": {{
#                                         "importDate": null,
#                                         "caseNumber": null
#                                     }},
#                                     "newDate": 1712668671435,
#                                     "timeToDetect": 0
#                                 }},
#                                 {{
#                                     "_id": "~8569048",
#                                     "_type": "Alert",
#                                     "_createdBy": "am@onzo.local",
#                                     "_createdAt": 1712316756734,
#                                     "type": "test",
#                                     "source": "python",
#                                     "sourceRef": "1",
#                                     "title": "Test From CrewAI",
#                                     "description": "description ...",
#                                     "severity": 3,
#                                     "severityLabel": "HIGH",
#                                     "date": 1712316756728,
#                                     "tags": [],
#                                     "tlp": 2,
#                                     "tlpLabel": "AMBER",
#                                     "pap": 2,
#                                     "papLabel": "AMBER",
#                                     "follow": true,
#                                     "customFields": [],
#                                     "observableCount": 0,
#                                     "status": "New",
#                                     "stage": "New",
#                                     "extraData": {{
#                                         "importDate": null,
#                                         "caseNumber": null
#                                     }},
#                                     "newDate": 1712316756732,
#                                     "timeToDetect": 0
#                                 }}
#                             ] 
#                            """),
#     agent=soc_manager
#     #tools=[thehive_alerts]
# )


In [11]:
tk_thehive_input = Task(
  description=dedent(f"""USE THE AVAILABLE TOOL and verify alerts for triage and analysis.
                        Use the tool provided and grab the alerts in json format for triage by the Triage Specialist.
                        """),
  expected_output = dedent(f"""Security alerts to be analyzed."""),
  agent=soc_manager
)


tk_triage_alerts = Task(
  description=dedent(f"""In context of cybersecurity continuous monitoring. Analyze the alerts provided by the SOC Manager.
                     The current time is {datetime.now()}. As a Triage Specialist, your primary responsibility is to efficiently evaluate and prioritize security alerts and events, 
                     identifying potential incidents that require further investigation and response. In this task, you will be analyzing a series of security events.
                     
                     Steps:
                        Review the events severity;
                        Analyze the collected data and events, looking for patterns, anomalies, or indicators of potential threats or incidents related to the research topic.
                        
                     For each identified event or alert: 
                        a. Assess the criticality and potential impact based on our OWN organization's risk assessment criteria. 
                        b. Determine if the event is a false positive or a justified alert requiring further investigation. 
                        c. Prioritize the events based on their criticality and potential impact. 
                        d. Enrich the events with relevant contextual information, such as source IP addresses, user accounts, timestamps, and any additional data that may aid in the investigation.

                    Document your findings and prioritized events in a concise report, following the established reporting format.
  
                      For events or alerts deemed critical or requiring further investigation, escalate them to Senior Cybersecurity SOC Analyst, 
                      providing detailed documentation and context to facilitate efficient incident response.
                      
                      Continuously monitor the SIEM and other monitoring tools for any new events or alerts related to the research topic, repeating the triage and analysis process as necessary.
                      
                      Deliverable:
                      
                      A comprehensive report detailing the triaged events, their prioritization, and any critical events escalated for further investigation. 
                      The report should be structured in a clear and concise manner, allowing the Senior Cybersecurity SOC Analyst to quickly understand the identified events and their potential impact.
                      
                      Note: 
                      As a Triage Specialist, you play a crucial role in the early stages of incident detection and response. 
                      Your attention to detail, analytical skills, and ability to prioritize events based on their criticality are essential in ensuring that potential threats are promptly identified and addressed.
                      """),
  expected_output=dedent(f"""A comprehensive analysis detailing the triaged events, their prioritization, and any critical events escalated for further investigation. 
                         Make sure that the timestamp is inserted into each alert reviewed result. 
                         Review the alert severity and return your sound judgement based on your expertise about the real severity level."""),
  agent=triage_specialist
  #context=[tk_thehive_input]
)

tk_triage_review = Task(
  description=dedent(f"""REVIEW the report created by the Triage Specialist.
                     Thoroughly investigate and analyze critical security alerts and events escalated by the Triage Specialist. In this task, you will be reviewing and assessing a prioritized list of events related to a specific topic of research.
                     
                     Make SURE to validate and double check the severity level assigned by the Triage Specialist and adjust as needed.
                     
                     Steps:
  
                      Review the comprehensive report provided by the Triage Specialist, which details the triaged events, their prioritization, and any critical events escalated for further investigation.
                      Assess the prioritized list of events and alerts, focusing on the critical events escalated for your review.
                      For each critical event or alert: 
                        a. Conduct an in-depth analysis, leveraging various sources of data and intelligence, such as security logs, network traffic captures, threat intelligence feeds, and any other relevant resources. 
                        b. Identify potential indicators of compromise (IOCs) and correlate them with known tactics, techniques, and procedures (TTPs) used by threat actors. 
                        c. Utilize industry-recognized frameworks like MITRE ATT&CK to map the observed TTPs and develop appropriate detection and response strategies. 
                        d. Collaborate with other security teams, such as incident response, forensics, and threat intelligence, to gather additional insights and context related to the event. 
                        e. Assess the potential impact and risk associated with the event, considering factors such as data sensitivity, system criticality, and potential business disruption.
                      
                      Based on your analysis, determine the appropriate course of action for each critical event, which may include: 
                        a. Initiating an incident response process to contain and mitigate the potential threat. 
                        b. Implementing additional security controls or countermeasures to prevent similar incidents in the future. 
                        c. Updating security policies, procedures, or configurations to enhance the organization's security posture. 
                        d. Providing recommendations for remediation or further investigation.
                      
                      Document your findings, analysis, and recommended actions in a comprehensive report, following established reporting protocols.
                      Communicate your findings and recommendations to relevant stakeholders, such as the SOC Manager, security leadership, and other cross-functional teams, as necessary.
                      Continuously monitor the situation and provide updates or further analysis as new information becomes available.

                      Deliverable:
                      
                      A detailed report documenting your analysis of the critical events, identified IOCs and TTPs, potential impact and risk assessment, recommended actions, and any additional insights or recommendations related to the specific topic of research. This report will serve as a crucial resource for the organization's security decision-making and incident response efforts.
                      
                      Note: 
                      
                      As a Senior Cybersecurity SOC Analyst, your expertise in threat analysis, incident response, and security best practices is ESSENTIAL in ensuring that potential threats are effectively mitigated, and the organization's security posture is continuously strengthened.
                      
                      Ensure that your final answer MUST saved to a markdown."""),
  expected_output=dedent(f"""A detailed report documenting your analysis of the critical events, identified IOCs and TTPs, potential impact and risk assessment, recommended actions, 
                         and any additional insights or recommendations related to the specific topic of research. 
                         """),
  agent=senior_soc_analyst
  #context=[tk_triage_alerts]
)

# tk_augument_threat_intell = Task(
#   description=dedent(f"""Taking the post created by the writer, take this and save it to a markdown file.
#                      Your final answer MUST be a response must be showing that the file was saved ."""),
#   expected_output='A saved file name',
#   agent=threat_intell
# )

tk_write_report = Task(
  description=dedent(f"""Provide comprehensive threat intelligence and analysis to support the organization's cybersecurity efforts. 
                     In this task, you will be generating a detailed threat intelligence report based on the critical events and alerts investigated by the Senior Cybersecurity SOC Analyst, 
                     as well as gathering additional insights from various sources.
  
                      Steps:
                        
                        1. Review the detailed report provided by the Senior Cybersecurity SOC Analyst, which documents the analysis of critical events, identified IOCs and TTPs, potential impact and risk assessment, and recommended actions.
                        2. Conduct thorough research and gather intelligence from various sources, including:
                          a. Threat intelligence feeds and repositories (e.g., MISP, OTX, VirusTotal)
                          b. Security research publications and industry reports
                          c. Dark web forums and online communities
                          d. Open-source intelligence (OSINT) sources
                          e. Web searches and online resources
                        
                        3. For each critical event or alert, perform the following:
                          a. Correlate the identified IOCs and TTPs with known threat actor groups, campaigns, or malware families.
                          b. Analyze the potential motivations, capabilities, and historical operations of the associated threat actors.
                          c. Assess the prevalence and potential impact of the identified vulnerabilities, if applicable.
                          d. Evaluate the relevance and applicability of published mitigation strategies, detection rules, or security advisories.

                        4. Leverage industry-recognized frameworks like MITRE ATT&CK and OWASP Threat Modeling to contextualize the observed TTPs and vulnerabilities within a broader attack lifecycle or application security context.
                        5. Compile your findings and intelligence into a comprehensive threat intelligence report, following a well-structured format that includes:
                          a. Executive summary
                          b. Overview of critical events and alerts
                          c. Threat actor profiles and motivations
                          d. Detailed analysis of IOCs, TTPs, and vulnerabilities
                          e. Mitigation strategies and recommended actions
                          f. Emerging threats and trends relevant to the organization

                        6. Incorporate visualizations, graphs, or diagrams to enhance the clarity and understanding of the report's content.
                        7. Collaborate with the Senior Cybersecurity SOC Analyst and other security teams to validate and refine your findings and recommendations.
                        8. Present and discuss the threat intelligence report with relevant stakeholders, such as the SOC Manager, security leadership, and other cross-functional teams, as necessary.

                      Deliverable:
                      
                      A comprehensive threat intelligence report that provides in-depth analysis and actionable intelligence related to the critical events and alerts investigated by the Senior SOC Analyst. The report should leverage various threat intelligence sources, incorporate industry-recognized frameworks, and provide valuable insights to support the organization's security decision-making and incident response efforts.

                      Note: As a Threat Intelligence, Threat Hunting, and Vulnerability Expert, your ability to synthesize information from diverse sources, analyze complex threats, 
                      and communicate findings effectively is crucial in enhancing the organization's overall security posture and preparedness against emerging threats.
                      """),
  expected_output=dedent(f"""A comprehensive review of the alerts reviewed by the Senior Cybersecurity SOC Analyst and
                         generate threat intelligence report that provides in-depth analysis and actionable intelligence related to the critical events and alerts. 
                         The report should leverage various threat intelligence sources, incorporate industry-recognized frameworks, 
                         and provide valuable insights to support the organization's security decision-making and incident response efforts."""),
  agent=threat_intell
  #tools=[search_tool, save_content],
  #context=[tk_triage_review, tk_triage_alerts]
)

In [12]:
nebuchadnezzar = Crew(
    agents=[soc_manager,
            triage_specialist,
            senior_soc_analyst,
            threat_intell],
    tasks=[tk_thehive_input,
           tk_triage_alerts,
           tk_triage_review,
           tk_write_report],
    verbose=2,
    process=Process.hierarchical,
    memory=True,
    full_output=True,
    share_crew=False,
    manager_llm=morpheus_gemma,
    #output_log_file=True,
    step_callback=lambda x: print_agent_output(x,"CyberCrew Run")
)

In [13]:
# Kick off the crew's work
soc_run = nebuchadnezzar.kickoff()
# Print the results
# print("Nebuchadnezzar Crew | Security Analysis Results:")
# print(soc_run)

with open ('crewai_output.md', 'w') as file:
    file.write("## Nebuchadnezzar Crew | Security Analysis Results: \n")
    file.write("```\n")  # Start of code block in markdown
    file.write(str(soc_run))
    file.write("\n```\n")  # End of code block in markdown

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: USE THE AVAILABLE TOOL and verify alerts for triage and analysis.
                        Use the tool provided and grab the alerts in json format for triage by the Triage Specialist.



> Entering new CrewAgentExecutor chain...
Action: Ask question to co-worker
Action Input: {
    "coworker": "SOC Manager",
    "question": "Can you provide me with the current list of security alerts in JSON format? We need to triage and analyze them as soon as possible.",
    "context": "As Crew Manager, I want to ensure that the team is informed about the latest security alerts. I need a comprehensive list in JSON format for efficient triaging by the Triage Specialist."
}

> Entering new CrewAgentExecutor chain...
Action: thehive_alerts
Action Input: {'severity': 2} 

[{'_id': '~8352', '_type': 'Alert', '_createdBy': 'am@onzo.local', '_createdAt': 1713100557701, 'type': 'wazuh_alert', 'source': 'wazuh', 'sourceRef': '941d14', 'title': 'SCA